# DEPLOY MODEL

In [1]:
# import necessary modules

import pandas as pd
import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
import os
import sys
import glob
import pickle

import warnings
import tqdm
import pandas as pd
import librosa
import librosa.display as display
import biosppy
import scipy.signal as sig
from scipy.stats import zscore
from scipy.io import wavfile
from scipy.fft import fft,fftfreq
from scipy.fftpack import fft

warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib as mlp
mlp.rc("xtick",labelsize=10)
mlp.rc("ytick",labelsize=10)
mlp.rc("axes",labelsize=11)
plt.rcParams["figure.figsize"] = [11,5]
plt.rcParams["figure.dpi"] = 100

import tensorflow as tf
import sklearn
import tensorflow.keras as keras
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

CURR_DIR = os.getcwd()

Using TensorFlow backend.


In [10]:
CURR_DIR = os.getcwd()

MAIN_DIR = "."
if os.path.basename(os.getcwd())!="Silent-Interface-for-IOT-Devices":
    os.chdir("..")

PICKLE_DIR = os.path.join(MAIN_DIR,"pickles")
MODEL_DIR = os.path.join(MAIN_DIR,"models")
FONT_DIR = os.path.join(MAIN_DIR,"fonts")
TEST_DIR = os.path.join(MAIN_DIR,"test")

SCALE_FACTOR = 0.022351744455307063
# dataset definitions
SENTENCES =["अबको समय सुनाउ","एउटा सङ्गित बजाउ","आजको मौसम बताउ","बत्तिको अवस्था बदल","पङ्खाको स्तिथी बदल"]
LABELS = np.array(SENTENCES)[[3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0]]


ORDERED_SENTENCES =["अबको समय सुनाउ","आजको मौसम बताउ","एउटा सङ्गित बजाउ","पङ्खाको स्तिथी बदल","बत्तिको अवस्था बदल"]
# ordered
# TEST_LABELS = np.array([4.0, 1.0, 2.0, 3.0, 0.0, 4.0, 1.0, 2.0, 3.0, 0.0, 4.0, 1.0, 2.0, 3.0, 0.0]) #real dataset


# unordered
# TEST_LABELS = np.array([0,1,2,0,3,1,0,3,0,0,1,1,3,3,4,4,2,3,1,2,2,2,4,4,4]) * 1.0 #sample dataset

# ordered
# TEST_LABELS = np.array([0.0, 2.0, 1.0, 0.0, 4.0, 2.0, 0.0, 4.0, 0.0, 0.0, 2.0, 2.0, 4.0, 4.0, 3.0, 3.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0]) #sample dataset


In [3]:
def RNN(INPUT_SHAPE,DROPOUT=0.3,learning_rate=0.0003,activation="relu",neurons=64,K_regulizer=0.001):

    model = keras.models.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv1D(64,(3),activation="relu",input_shape=INPUT_SHAPE,
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling1D((3), strides=(2), padding='same'))


    model.add(tf.keras.layers.Conv1D(32, (3), activation='relu',
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling1D((3), strides=(2), padding='same'))

    # 3rd conv layer
    model.add(tf.keras.layers.Conv1D(32, (2), activation='relu',
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling1D((2), strides=(2), padding='same'))


    model.add(LSTM(units = 50, return_sequences = True))
    tf.keras.layers.Dropout(0.3)
    model.add(LSTM(units = 50, return_sequences = True))
    tf.keras.layers.Dropout(0.3)
    model.add(LSTM(units = 50, return_sequences = True))
    tf.keras.layers.Dropout(0.3)

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    tf.keras.layers.Dropout(0.3)

    # softmax output layer
    model.add(tf.keras.layers.Dense(5, activation='softmax'))
        
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    print(model.summary())
    return model


In [4]:
model = RNN(INPUT_SHAPE=(1648,8))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1646, 64)          1600      
_________________________________________________________________
batch_normalization (BatchNo (None, 1646, 64)          256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 823, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 821, 32)           6176      
_________________________________________________________________
batch_normalization_1 (Batch (None, 821, 32)           128       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 411, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 410, 32)           2

In [6]:
# load model
model.load_weights(os.path.join(MODEL_DIR,'1DCNN-3alyer-LSTM-3alyer-30EPOCH.h5'))

## PARSE DATA

In [7]:
def parser(files,NORMALIZE=True,DEPLOY=False):
    
    """
    parser function to extract utterances from .txt file and store them in a dictionary
    """
    
    # PERCENTILES FOR LENGTH NORMALIZATION
    if(NORMALIZE==True):
        percentile_95 = 1602
        percentile_97 = 1614
        percentile_99 = 1648
        percentile_100 = 1875
    else:
        percentile_95 = percentile_97 = percentile_99 = percentile_100 = -1
        
    dataset = {"data":[], "speaker":[],"session":[],"labels":[]}
    
    def get_data(file):
        
        signal = read_data(file)
        
        if(DEPLOY==True):
            dataset["data"].extend(signal)
            return
        
#         if(len(signal)!=25):
#             return
        
        session = file.split("/")[-2]
        speaker = file.split("/")[-3]
        
        dataset["data"].extend(signal)
        dataset["speaker"].extend([speaker]*len(signal))
        dataset["session"].extend([session]*len(signal))
        dataset["labels"].extend(LABELS)
        
    
    def read_data(file):
        f = open(file, 'r')
        contents = map(lambda x : x.strip(), f.readlines())
        #the file starts with '%' and some instruction before data and removing these data 
        frames_original = list(filter(lambda x : x and x[0] != '%', contents))[1:]
        #the data row contains channels info digital trigger and accelerometer info separated by comma
        frames_original = list(map(lambda s : list(map( lambda ss: ss.strip(), s.split(','))), frames_original))
        # (8 channels) + digital triggers
        # the digital trigger is in a[16], used to indicate the utterance
        frames = list(map(lambda a: list(map(float, a[1:9])) + [float(a[16])] , frames_original))
        frames = np.array(frames)
        indices = []
        signal = []
        for index,f in enumerate(frames[:,-1]):
            if(bool(f) ^ bool(frames[(index+1) if ((index+1)<len(frames)) else index,-1]) ):
                indices.append(index)
                if len(indices)>1 and len(indices)%2==0:
                    frame_len = indices[len(indices)-1] - indices[len(indices)-2]
                    if(frame_len<percentile_99):
                        pad = int(np.ceil((percentile_99 - frame_len)/2))
                    else:
                        pad = 0
                    left_pad = indices[len(indices)-2] - pad
                    right_pad = indices[len(indices)-1] + pad
                    a_frame = (frames[left_pad:right_pad,:-1])[:percentile_99]
                    signal.append(a_frame)
    
        # convert to microVolts and return
        return np.array(signal)*SCALE_FACTOR
        
#     for file,i in zip(files,tqdm.tqdm(range(1,len(files)+1),desc="PARSING DATA")):
#         get_data(file)


    for file in files:
        get_data(file)
    
    return dataset

## PROCESS SIGNAL

In [8]:
def signal_pipeline(data,RICKER=False):

    """
    CORRECT DC DRIFT --> CORRECT DC BIAS --> SMOOTHING SIGNAL --> NORMALIZE DATA --> FILTER DATA 	
    """
    filter_data = []

    def digital_filter(data,HPF=0.5,LPF=10,H_ORDER=4,L_ORDER=4,SR=250):
        """
        HPF --> NOTCH --> LPF --> RICKER CONVOLUTION
        """

        # highpass filter
        f_signal = biosppy.signals.tools.filter_signal(data,ftype="butter",band="highpass",order=H_ORDER,sampling_rate=SR,frequency=HPF)
        # notch filter
        b,a = sig.iirnotch(50,30,SR)
        f_signal = sig.lfilter(b,a,f_signal[0])

        # lowpass filter
        f_signal = biosppy.signals.tools.filter_signal(f_signal,ftype="butter",band="lowpass",order=L_ORDER,sampling_rate=SR,frequency=LPF)

        if(RICKER==True):
            # RICKER CONVOLUTION TO REMOVE HEARTBEAT ARTIFACTS
            ricker_width = 35 * SR // 250
            ricker_sigma = 4.0 * SR / 250
            ricker = sig.ricker(ricker_width,ricker_sigma)
            # normalize ricker
            ricker = np.array(ricker, np.float32) / np.sum(np.abs(ricker))
            convolution = sig.convolve(f_signal[0],ricker,mode="same")
            return (f_signal[0]-2*convolution)

        return f_signal[0]

    def process_signal(data):
        f_data = []
        for i in range(8):
            # correction of DC drift
            c_data = data[:,i]- data[0,i]

            # correct DC bias
            c_data = c_data - np.mean(c_data)

            # normalize and filter data
            c_data = digital_filter(c_data)
            f_data.append(c_data)

        return np.array(f_data).T

#     for d,i in zip(data,tqdm.tqdm(range(1,len(data)+1),desc="PROCESSING DATA: ")):
#         temp_data = process_signal(d)
#         filter_data.extend([temp_data])

    for d in data:
        temp_data = process_signal(d)
        filter_data.extend([temp_data])
        
    return np.array(filter_data)


In [8]:
print("Fetching filtered data from pickle file ...")
all_data_filtered = pickle.load(open(os.path.join(PICKLE_DIR,"data_dict_filtered.pickle"),"rb"))
print("Done!")

Fetching filtered data from pickle file ...
Done!


In [9]:
def prepare_dataset(all_data_filtered,K_FOLD=False):
    data = all_data_filtered["data"]
    labels = all_data_filtered["labels"]

    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels).astype('float64')

    print(encoded_labels[:10])
    print(all_data_filtered["labels"][:10])

    X = np.array(data)
    Y = encoded_labels
    
    if(K_FOLD==True):
        return X,Y
    
    splitter = StratifiedShuffleSplit(n_splits=1,test_size=0.1, random_state=42)

    # train test split
    train_id, test_id = next(splitter.split(X,Y))
    X_train,y_train,X_test,y_test = X[train_id],Y[train_id],X[test_id],Y[test_id]

    
    # train val split
    train_id, test_id = next(splitter.split(X_train,y_train))
    X_train,y_train,X_val,y_val = X_train[train_id],y_train[train_id],X_train[test_id],y_train[test_id]
    print("Shape of data instance: ", X_train[0].shape)
    print("Shape of Training data: ",X_train.shape)
    print("Shape of Testing data: ",X_test.shape)

    return X_train,y_train,X_val,y_val,X_test,y_test

In [10]:
X_train,y_train,X_val,y_val,X_test,y_test = prepare_dataset(all_data_filtered)


[4. 1. 2. 3. 0. 4. 1. 2. 3. 0.]
['बत्तिको अवस्था बदल', 'आजको मौसम बताउ', 'एउटा सङ्गित बजाउ', 'पङ्खाको स्तिथी बदल', 'अबको समय सुनाउ', 'बत्तिको अवस्था बदल', 'आजको मौसम बताउ', 'एउटा सङ्गित बजाउ', 'पङ्खाको स्तिथी बदल', 'अबको समय सुनाउ']
Shape of data instance:  (1648, 8)
Shape of Training data:  (5284, 1648, 8)
Shape of Testing data:  (653, 1648, 8)


In [11]:
model.evaluate(X_val,y_val)

19/19 [==============================] - 2s 94ms/step - loss: 0.4219 - accuracy: 0.8895


[0.42186692357063293, 0.8894557952880859]

In [9]:
def deploy_pipeline(file):
    data = parser(file,DEPLOY=True)["data"]
    data = signal_pipeline(data)
    return data

In [44]:
# ordered
TEST_LABELS = np.array([4.0, 1.0, 2.0, 3.0, 0.0, 4.0, 1.0, 2.0, 3.0, 0.0, 4.0, 1.0, 2.0, 3.0, 0.0]) #real dataset

for i in range(1,11):
    data = deploy_pipeline(["test"+str(i)+".txt"])
    if(len(data)>15):
        continue
    print(model.evaluate(data,TEST_LABELS))

1/1 [==============================] - 0s 2ms/step - loss: 0.7478 - accuracy: 0.8000
[0.7477979063987732, 0.800000011920929]
1/1 [==============================] - 0s 1ms/step - loss: 1.4213 - accuracy: 0.6667
[1.4213029146194458, 0.6666666865348816]
1/1 [==============================] - 0s 1ms/step - loss: 1.3596 - accuracy: 0.5333
[1.3595722913742065, 0.5333333611488342]
1/1 [==============================] - 0s 1ms/step - loss: 0.7502 - accuracy: 0.8000
[0.7502457499504089, 0.800000011920929]
1/1 [==============================] - 0s 1ms/step - loss: 0.3219 - accuracy: 0.9333
[0.321938157081604, 0.9333333373069763]
1/1 [==============================] - 0s 1ms/step - loss: 0.3883 - accuracy: 0.9333
[0.38830137252807617, 0.9333333373069763]
1/1 [==============================] - 0s 1ms/step - loss: 0.1872 - accuracy: 0.9333
[0.18722939491271973, 0.9333333373069763]
1/1 [==============================] - 0s 2ms/step - loss: 0.8208 - accuracy: 0.8667
[0.8208482265472412, 0.86666667461

In [49]:
# unordered
# TEST_LABELS = np.array([0,1,2,0,3,1,0,3,0,0,1,1,3,3,4,4,2,3,1,2,2,2,4,4,4]) * 1.0 #sample dataset

# ordered
TEST_LABELS = np.array([0.0, 2.0, 1.0, 0.0, 4.0, 2.0, 0.0, 4.0, 0.0, 0.0, 2.0, 2.0, 4.0, 4.0, 3.0, 3.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0]) #sample dataset

for i in range(1,11):
    data = deploy_pipeline(["ttest"+str(i)+".txt"])
    if(len(data)>25):
        continue
    print(model.evaluate(data,TEST_LABELS))

1/1 [==============================] - 0s 1ms/step - loss: 3.4731 - accuracy: 0.2400
[3.4731409549713135, 0.23999999463558197]
1/1 [==============================] - 0s 1ms/step - loss: 3.6576 - accuracy: 0.2400
[3.6575863361358643, 0.23999999463558197]
1/1 [==============================] - 0s 1ms/step - loss: 2.8729 - accuracy: 0.4000
[2.872880458831787, 0.4000000059604645]
1/1 [==============================] - 0s 1ms/step - loss: 2.6409 - accuracy: 0.3200
[2.6409058570861816, 0.3199999928474426]
1/1 [==============================] - 0s 1ms/step - loss: 2.5988 - accuracy: 0.4400
[2.5987653732299805, 0.4399999976158142]
1/1 [==============================] - 0s 1ms/step - loss: 5.2730 - accuracy: 0.0800
[5.272961616516113, 0.07999999821186066]
1/1 [==============================] - 0s 1ms/step - loss: 3.3351 - accuracy: 0.1200
[3.335120916366577, 0.11999999731779099]
1/1 [==============================] - 0s 1ms/step - loss: 3.1124 - accuracy: 0.2400
[3.112440824508667, 0.2399999946

In [22]:
data = deploy_pipeline([os.path.join(TEST_DIR,"A.txt")])
predictions = np.array(ORDERED_SENTENCES)[list(map(np.argmax,model.predict_proba(data)))]
print(*predictions, sep = "\n")

बत्तिको अवस्था बदल
पङ्खाको स्तिथी बदल
आजको मौसम बताउ
आजको मौसम बताउ
आजको मौसम बताउ
आजको मौसम बताउ
पङ्खाको स्तिथी बदल
पङ्खाको स्तिथी बदल
अबको समय सुनाउ
आजको मौसम बताउ


In [23]:
data = deploy_pipeline([os.path.join(TEST_DIR,"B.txt")])
predictions = np.array(ORDERED_SENTENCES)[list(map(np.argmax,model.predict_proba(data)))]
print(*predictions, sep = "\n")

आजको मौसम बताउ
एउटा सङ्गित बजाउ
आजको मौसम बताउ
आजको मौसम बताउ
आजको मौसम बताउ
बत्तिको अवस्था बदल
आजको मौसम बताउ
आजको मौसम बताउ
आजको मौसम बताउ
एउटा सङ्गित बजाउ
